<a href="https://colab.research.google.com/github/nguyen-nhat-mai/methane-leak-detection/blob/main/Web_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build web app for classifier model

In [58]:
!pip install streamlit -q

In [98]:
import pandas as pd

In [105]:
df = pd.read_csv("metadata.csv")
#df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df.head()

,date,id_coord,plume,set,lat,lon,coord_x,coord_y,path
0,20230223,id_6675,yes,train,31.528750,74.330625,24,47,images/plume/20230223_methane_mixing_ratio_id_...
1,20230103,id_2542,yes,train,35.538000,112.524000,42,37,images/plume/20230103_methane_mixing_ratio_id_...
2,20230301,id_6546,yes,train,21.060000,84.936667,58,15,images/plume/20230301_methane_mixing_ratio_id_...
3,20230225,id_6084,yes,train,26.756667,80.973333,28,62,images/plume/20230225_methane_mixing_ratio_id_...
4,20230105,id_2012,yes,train,34.800000,40.770000,59,44,images/plume/20230105_methane_mixing_ratio_id_...


Reference:

https://towardsdatascience.com/create-an-image-classification-web-app-using-pytorch-and-streamlit-f043ddf00c24#2b4c
https://www.youtube.com/watch?v=NEhrkeF2o_M

In [115]:
# Write the web app
%%writefile app.py
import streamlit as st
from PIL import Image
from torchvision import models, transforms
import torch
import time
import pandas as pd
import base64
from io import BytesIO
import altair as alt

#---------------------- TO UPDATE WITH FINAL MODEL-----------------------#

def predict(image_path):
    # Load model
    best_model = models.resnet101(pretrained=True)
    # Define transformation
    transform = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(
      mean=[0.485, 0.456, 0.406],
      std=[0.229, 0.224, 0.225]
      )])
    # Load data
    img = Image.open(image_path)
    batch_t = torch.unsqueeze(transform(img), 0)
    # Do inference
    best_model.eval()
    out = best_model(batch_t)
    # Load all of the classes => Yes/ No for methane case
    with open('imagenet_classes.txt') as f:
        classes = [line.strip() for line in f.readlines()]
    # Calculate the probability
    prob = torch.nn.functional.softmax(out, dim=1)[0] * 100
    _, indices = torch.sort(out, descending=True)
    # Return the classes and corresponding probability
    return [(classes[idx], prob[idx].item()) for idx in indices[0][:1]]

# ------------------------------WEB APP----------------------------------#

# Define a function to create a download link for a given DataFrame
def download_link(df, filename, text):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode('utf-8')).decode()  # base64 encoding
    href = f"data:text/csv;base64,{b64}"
    return f'<a href="{href}" download="{filename}">{text}</a>'

# Get dictionary of geolocation for id_coord
geo_map = pd.read_csv("metadata.csv")
geo_dict = {row['id_coord']: (row['lat'], row['lon']) for _, row in geo_map.iterrows()}

# Start the display on web app
st.set_option('deprecation.showfileUploaderEncoding', False)
st.title("Detect Methane Leaks")
st.write("")

tabs = ["Prediction on uploaded images", "Leak detection map", "Leak track over time by id_coord", "Prediction Detail"]
selected_tab = st.sidebar.radio("Select a tab", tabs)

if selected_tab == "Prediction on uploaded images":
    file_up = st.file_uploader("Upload your images", type=["jpg", "jpeg", "png"], accept_multiple_files=True)
    if file_up is not None:
        st.subheader("Prediction on uploaded images")
        all_predictions = pd.DataFrame() # create an empty DataFrame to store all predictions
        for img_file in file_up:
            image = Image.open(img_file)
            st.image(image, caption=img_file.name, use_column_width=True)
            st.write("")
            with st.spinner('Predicting...'):
                predictions_df = predict(img_file)
                # Append the current predictions to the DataFrame along with the id_coord, date and geolocation
                id_coord = img_file.name.split(".")[0][-7:]
                date = img_file.name.split("_")[0]
                latitude = geo_dict[id_coord][0]
                longitude = geo_dict[id_coord][1]
                predictions_df = [(id_coord,date,latitude,longitude,)+predictions_df[0]]
                all_predictions = all_predictions.append(predictions_df, ignore_index=True)
                # Print out the prediction labels with probability
                st.write("Prediction:", predictions_df[0][4], "-   Probability (%): ", round(predictions_df[0][5],0))
        if not all_predictions.empty:
            all_predictions.columns = ['Id_coord', "Date","Latitude","Longitude",'Label', 'Probability (%)']
            all_predictions.to_csv("predictions.csv", index=False)

if selected_tab == "Leak detection map":
    st.subheader("Leak detection map")
    st.write("wait")

if selected_tab == "Leak track over time by id_coord":
    all_predictions = pd.read_csv("predictions.csv")
    all_predictions['Date'] = pd.to_datetime(all_predictions['Date'], format='%Y%m%d')
    st.subheader("Leak track over time by id_coord")
    id_coord_list = list(all_predictions['Id_coord'].unique())
    selected_id_coord = st.sidebar.selectbox("Select an id_coord:", id_coord_list)
    filtered_predictions = all_predictions.loc[all_predictions['Id_coord'] == selected_id_coord]
    chart = alt.Chart(filtered_predictions).mark_line().encode(x='Date',y='Probability (%)')
    st.altair_chart(chart, use_container_width=True)

if selected_tab == "Prediction Detail":
    all_predictions = pd.read_csv("predictions.csv")
    st.subheader("Prediction Detail")
    all_predictions.to_csv("predictions.csv", index=False)
    st.markdown(download_link(all_predictions, "prediction.csv", "Download CSV"), unsafe_allow_html=True)
    st.write("")
    st.write(all_predictions)
        

Overwriting app.py


In [116]:
# Run app.py and made available on a local URL
!streamlit run app.py & npx localtunnel --port 8501

[############......] - refresh-package-json:localtunnel: timing action:finalize

npx: installed 22 in 4.251s

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.139.171.39:8501

your url is: https://wet-shirts-camp.loca.lt
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/content/app.py:78: FutureWarning: The frame.append method is deprecated and will 